In [ ]:
"""
This single file includes every function it depends on, so it can run without
importing from src/. It mirrors:
1. POST /compare-qps  -> `pair` subcommand (skill/task/NSQF/composite scores)
2. POST /qp-similarity-scores -> `matrix` subcommand (sector-to-sector matrix)
"""
from __future__ import annotations

import argparse
import json
import pickle
from dataclasses import dataclass, field
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

# Public API functions that callers can import directly
__all__ = [
    "run_pair_command",
    "run_matrix_command",
    "get_percentage_match",
    "get_task_similarity",
    "get_nsqf_level_similarity",
    "get_composite_similarity",
    "score_matrix",
]

# ------------------------------------------------------------------------------------
# User-facing helpers (defined first so they can be imported easily)
# ------------------------------------------------------------------------------------


def run_pair_command(lhs_qp_id: str, rhs_qp_id: str) -> dict[str, Any]:
    """
    Compute skill, task, NSQF, and composite similarity scores for two QPs.
    Mirrors the /compare-qps endpoint.
    """
    lhs_qp = get_qp_by_id(lhs_qp_id)
    rhs_qp = get_qp_by_id(rhs_qp_id)
    if not lhs_qp or not rhs_qp:
        missing = lhs_qp_id if not lhs_qp else rhs_qp_id
        raise ValueError(
            f"QP '{missing}' not found. Ensure data/qp-listings files are available."
        )

    skill_similarity = get_percentage_match(lhs_qp_id, rhs_qp_id)
    task_similarity = get_task_similarity(lhs_qp_id, rhs_qp_id)

    lhs_level = float(lhs_qp.nsqf_level if lhs_qp.nsqf_level is not None else 5.0)
    rhs_level = float(rhs_qp.nsqf_level if rhs_qp.nsqf_level is not None else 5.0)
    same_sector = (
        lhs_qp.sector.id == rhs_qp.sector.id
        if lhs_qp.sector and rhs_qp.sector
        else False
    )
    nsqf_similarity = get_nsqf_level_similarity(lhs_level, rhs_level, same_sector)

    composite = get_composite_similarity(
        skill_similarity, task_similarity, nsqf_similarity
    )

    return {
        "lhs_qp_id": lhs_qp_id,
        "rhs_qp_id": rhs_qp_id,
        "similarity_score": skill_similarity,
        "task_similarity_score": task_similarity,
        "nsqf_level_similarity_score": nsqf_similarity,
        "composite_similarity": composite,
    }


def run_matrix_command(
    sector1: str,
    sector2: str,
    sub_sector1: Optional[str] = None,
    sub_sector2: Optional[str] = None,
    occupation1: Optional[str] = None,
    occupation2: Optional[str] = None,
    filter_levels: Optional[List[str]] = None,
    filter_technical: Optional[List[str]] = None,
) -> dict[str, Any]:
    """
    Compute the sector-to-sector similarity matrix.
    Mirrors the /qp-similarity-scores endpoint.
    """
    return score_matrix(
        sector1=sector1,
        sector2=sector2,
        sub_sector1=sub_sector1,
        sub_sector2=sub_sector2,
        occupation1=occupation1,
        occupation2=occupation2,
        filter_levels=filter_levels,
        filter_technical=filter_technical,
    )


# ------------------------------------------------------------------------------------
# Data models and loaders (parity with src.qps.utils but embedded for standalone use)
# ------------------------------------------------------------------------------------

DATA_DIR = Path("data")
QP_CSV_PATH = DATA_DIR / "qp-listings.csv"
TASK_JSON_DIR = DATA_DIR / "task-jsons"
ALL_L2_SKILLS = DATA_DIR / "l2_vector_74_esco.json"
SIMILARITY_MATRIX_PATH = DATA_DIR / "similarities" / "qp_similarities.pkl"

SKILL_SIMILARITY_WEIGHTED_AVG = 0.33
TASK_SIMILARITY_WEIGHTED_AVG = 0.33
NSQF_LEVEL_SIMILARITY_WEIGHTED_AVG = 0.33


@dataclass
class SubSector:
    id: Optional[str]
    name: Optional[str]


@dataclass
class Sector:
    id: Optional[str]
    name: Optional[str]
    sub_sectors: List[SubSector] = field(default_factory=list)


@dataclass
class Occupation:
    id: Optional[str]
    code: Optional[str]
    description: Optional[str]


@dataclass
class QP:
    code: str
    version: Optional[float]
    nsqf_level: Optional[float]
    job_role: str
    job_role_description: str
    sector: Sector
    occupation: Occupation
    technical: bool
    economic_sector: Optional[str]
    economic_sector_type: Optional[str]

    @property
    def _id(self) -> str:
        if self.version is not None:
            return f"{self.code.replace('/', '_')}_{self.version}"
        return self.code


def _ensure_dict(value) -> Optional[dict]:
    if value is None or value == "":
        return None
    if isinstance(value, dict):
        return value
    try:
        parsed = json.loads(value)
        if isinstance(parsed, dict):
            return parsed
    except Exception:
        return None
    return None


def _parse_sector(raw: Optional[str]) -> Sector:
    data = _ensure_dict(raw) or {}
    sub_sectors_raw = data.get("subSectors") or []
    sub_sectors = [
        SubSector(
            id=str(sub.get("subSectorID")) if sub.get("subSectorID") else None,
            name=sub.get("subSectorName"),
        )
        for sub in sub_sectors_raw
    ]
    return Sector(
        id=str(data.get("sectorID")) if data.get("sectorID") else None,
        name=data.get("sectorName"),
        sub_sectors=sub_sectors,
    )


def _parse_occupation(raw: Optional[str]) -> Occupation:
    obj = _ensure_dict(raw) or {}
    return Occupation(
        id=str(obj.get("occupationID")) if obj.get("occupationID") else None,
        code=obj.get("occupationCode"),
        description=obj.get("occupationDesc"),
    )


def _parse_param_desc(raw: Optional[str]) -> Optional[str]:
    obj = _ensure_dict(raw)
    if not obj:
        return None
    return obj.get("paramDesc")


def _parse_technical(raw: Optional[str]) -> bool:
    desc = (_parse_param_desc(raw) or "").strip().lower()
    return desc == "technical"


def _load_qp_records(csv_path: Path) -> list[dict]:
    df = pd.read_csv(csv_path)
    df = df.sort_values(by="version", ascending=False)
    df = df.drop_duplicates(subset=["qpCode"])
    df = df.replace({pd.NA: None, np.nan: None})
    df = df[df["matched_filename"].notna()]
    df = df[df["matched_filename"] != ""]
    return [row.to_dict() for _, row in df.iterrows()]


def _build_qp_from_row(row: dict) -> QP:
    return QP(
        code=row.get("qpCode"),
        version=row.get("version"),
        nsqf_level=row.get("nsqfLevel"),
        job_role=row.get("jobRole"),
        job_role_description=row.get("jobRoleDesc"),
        sector=_parse_sector(row.get("sectors")),
        occupation=_parse_occupation(row.get("occupation")),
        technical=_parse_technical(row.get("qpParamOne")),
        economic_sector=_parse_param_desc(row.get("qpParamTwo")),
        economic_sector_type=_parse_param_desc(row.get("qpParamThree")),
    )


_QP_CACHE: Optional[List[QP]] = None
_QP_LOOKUP: Dict[str, QP] = {}


def get_all_qps() -> List[QP]:
    global _QP_CACHE, _QP_LOOKUP
    if _QP_CACHE is None:
        qps = [_build_qp_from_row(row) for row in _load_qp_records(QP_CSV_PATH)]
        _QP_CACHE = qps
        _QP_LOOKUP = {qp._id: qp for qp in qps}
    return _QP_CACHE


def get_qp_by_id(qp_id: str) -> Optional[QP]:
    get_all_qps()
    return _QP_LOOKUP.get(qp_id)


def get_filtered_qps(
    sector: str,
    sub_sector: Optional[str] = None,
    occupation: Optional[str] = None,
    levels: Optional[List[str]] = None,
    technical: Optional[List[str]] = None,
) -> List[QP]:
    qps = [qp for qp in get_all_qps() if qp.sector.id == sector]
    if sub_sector:
        qps = [
            qp
            for qp in qps
            if qp.sector.sub_sectors
            and qp.sector.sub_sectors[0].id == sub_sector
        ]
    if occupation:
        qps = [qp for qp in qps if qp.occupation.id == occupation]
    if levels:
        qps = [
            qp
            for qp in qps
            if qp.nsqf_level is not None and str(int(qp.nsqf_level)) in levels
        ]
    if technical:
        qps = [
            qp
            for qp in qps
            if qp.technical is not None
            and ("Technical" if qp.technical else "Non-Technical") in technical
        ]
    return qps


# ------------------------------------------------------------------------------------
# Task data loading + task similarity logic
# ------------------------------------------------------------------------------------

_TASK_CACHE: Dict[str, dict] = {}
_L2_SKILLS_CACHE: Optional[List[str]] = None


def get_qp_task_file_name(qp_id: str) -> str:
    parts = qp_id.split("_")
    if len(parts) >= 3:
        version = "_".join(parts[2:])
        return f"{parts[0]}_{parts[1]}_{version}.json"
    return f"{qp_id}.json"


def get_qp_task_data(qp_id: str) -> dict:
    if qp_id in _TASK_CACHE:
        return _TASK_CACHE[qp_id]

    file_path = TASK_JSON_DIR / get_qp_task_file_name(qp_id)
    if not file_path.exists():
        _TASK_CACHE[qp_id] = {}
        return {}

    with open(file_path, "r", encoding="utf-8", errors="replace") as handle:
        data = json.load(handle)
    _TASK_CACHE[qp_id] = data
    return data


def calculate_skill_similarity(hierarchy1, hierarchy2, all_l2_skills) -> float:
    try:
        l1_names1 = {item.get("l1") for item in hierarchy1 if item.get("l1")}
        l1_names2 = {item.get("l1") for item in hierarchy2 if item.get("l1")}
        all_l1_dimensions = sorted(list(l1_names1.union(l1_names2)))

        similarity_l1 = 0.0
        if all_l1_dimensions:
            l1_dimension_map = {name: i for i, name in enumerate(all_l1_dimensions)}
            num_l1_dimensions = len(all_l1_dimensions)
            vector1_l1 = np.zeros(num_l1_dimensions)
            vector2_l1 = np.zeros(num_l1_dimensions)

            for item in hierarchy1:
                idx = l1_dimension_map.get(item.get("l1"))
                if idx is not None:
                    vector1_l1[idx] = item.get(
                        "norm_w_sum_of_all_tasks_under_this_l1", 0.0
                    )

            for item in hierarchy2:
                idx = l1_dimension_map.get(item.get("l1"))
                if idx is not None:
                    vector2_l1[idx] = item.get(
                        "norm_w_sum_of_all_tasks_under_this_l1", 0.0
                    )

            norm1_l1 = np.linalg.norm(vector1_l1)
            norm2_l1 = np.linalg.norm(vector2_l1)

            if norm1_l1 > 0 and norm2_l1 > 0:
                dot_product_l1 = np.dot(vector1_l1, vector2_l1)
                similarity_l1 = dot_product_l1 / (norm1_l1 * norm2_l1)

        l2_dimension_map = {skill: i for i, skill in enumerate(all_l2_skills)}
        num_l2_dimensions = len(all_l2_skills)
        vector1_l2 = np.zeros(num_l2_dimensions)
        vector2_l2 = np.zeros(num_l2_dimensions)

        for l1_item in hierarchy1:
            if "children" in l1_item and isinstance(l1_item["children"], list):
                for l2_item in l1_item["children"]:
                    l2_name = l2_item.get("l2")
                    if l2_name in l2_dimension_map:
                        idx = l2_dimension_map[l2_name]
                        vector1_l2[idx] = l2_item.get(
                            "norm_w_sum_of_all_tasks_under_this_l2", 0.0
                        )

        for l1_item in hierarchy2:
            if "children" in l1_item and isinstance(l1_item["children"], list):
                for l2_item in l1_item["children"]:
                    l2_name = l2_item.get("l2")
                    if l2_name in l2_dimension_map:
                        idx = l2_dimension_map[l2_name]
                        vector2_l2[idx] = l2_item.get(
                            "norm_w_sum_of_all_tasks_under_this_l2", 0.0
                        )

        similarity_l2 = 0.0
        norm1_l2 = np.linalg.norm(vector1_l2)
        norm2_l2 = np.linalg.norm(vector2_l2)

        if norm1_l2 > 0 and norm2_l2 > 0:
            dot_product_l2 = np.dot(vector1_l2, vector2_l2)
            similarity_l2 = dot_product_l2 / (norm1_l2 * norm2_l2)

        final_similarity = ((similarity_l1 + similarity_l2) / 2) * 100
        return final_similarity

    except Exception:
        return 0.0


def get_task_similarity(
    lhs_qp_id: str,
    rhs_qp_id: str,
    lhs_hierarchy=None,
    rhs_hierarchy=None,
) -> float:
    global _L2_SKILLS_CACHE

    if lhs_hierarchy is None:
        lhs_task_data = get_qp_task_data(lhs_qp_id)
        lhs_hierarchy = lhs_task_data.get("skill_hierarchy")
    if rhs_hierarchy is None:
        rhs_task_data = get_qp_task_data(rhs_qp_id)
        rhs_hierarchy = rhs_task_data.get("skill_hierarchy")

    if not lhs_hierarchy or not rhs_hierarchy:
        return 0.0

    if _L2_SKILLS_CACHE is None:
        if not ALL_L2_SKILLS.exists():
            return 0.0
        with open(ALL_L2_SKILLS, "r", encoding="utf-8") as handle:
            _L2_SKILLS_CACHE = json.load(handle)

    return calculate_skill_similarity(lhs_hierarchy, rhs_hierarchy, _L2_SKILLS_CACHE)


# ------------------------------------------------------------------------------------
# Similarity matrix handling (rewritten from src.similarity_utils)
# ------------------------------------------------------------------------------------


class SimilarityMatrix:
    def __init__(self, qp_codes: Optional[List[str]] = None):
        if qp_codes:
            self.qp_codes = qp_codes
            self.code_to_index = {code: i for i, code in enumerate(qp_codes)}
            self.matrix = np.ones((len(qp_codes), len(qp_codes)))
        else:
            self.qp_codes = []
            self.code_to_index = {}
            self.matrix = np.array([])

    def normalize_score(self, score: float) -> float:
        if score <= 0.0:
            return 0.0
        min_val = 0.35
        max_val = 0.85
        score_range = max_val - min_val
        normalized_scores = ((score - min_val) / score_range) * 100
        normalized_scores_clipped = max(0.0, min(100.0, normalized_scores))
        return round(normalized_scores_clipped, 2)

    def has_qp_code(self, qp_code: str) -> bool:
        return qp_code in self.code_to_index

    def get_similarity(self, qp1_code: str, qp2_code: str) -> float:
        if qp1_code not in self.code_to_index or qp2_code not in self.code_to_index:
            return 0.0
        i, j = self.code_to_index[qp1_code], self.code_to_index[qp2_code]
        return self.matrix[i, j]

    @classmethod
    def load(cls, filepath: Path) -> "SimilarityMatrix":
        with open(filepath, "rb") as handle:
            data = pickle.load(handle)

        instance = cls()
        if isinstance(data, pd.DataFrame):
            instance.qp_codes = data.index.tolist()
            instance.code_to_index = {
                code: i for i, code in enumerate(instance.qp_codes)
            }
            instance.matrix = data.values
        elif isinstance(data, dict):
            instance.qp_codes = data["qp_codes"]
            instance.code_to_index = data["code_to_index"]
            instance.matrix = data["matrix"]
        else:
            raise ValueError(
                f"Unsupported data format: {type(data)}. Expected DataFrame or dict."
            )
        return instance


_SIM_MATRIX: Optional[SimilarityMatrix] = None


def _get_similarity_matrix() -> SimilarityMatrix:
    global _SIM_MATRIX
    if _SIM_MATRIX is None:
        if not SIMILARITY_MATRIX_PATH.exists():
            raise FileNotFoundError(
                f"Similarity matrix not found at {SIMILARITY_MATRIX_PATH}"
            )
        _SIM_MATRIX = SimilarityMatrix.load(SIMILARITY_MATRIX_PATH)
    return _SIM_MATRIX


def get_percentage_match(lhs_qp_id: str, rhs_qp_id: str) -> float:
    sim_matrix = _get_similarity_matrix()
    raw_score = sim_matrix.get_similarity(lhs_qp_id, rhs_qp_id)
    return sim_matrix.normalize_score(raw_score)


def get_nsqf_level_similarity(
    job_a_level: float, job_b_level: float, is_same_sector: bool
) -> float:
    MIN_LEVEL = 1
    MAX_LEVEL = 8
    BETA = 5.5
    MAX_PENALTY = (MAX_LEVEL - MIN_LEVEL) * MAX_LEVEL + BETA * MAX_LEVEL

    job_a_level = max(MIN_LEVEL, min(MAX_LEVEL, job_a_level))
    job_b_level = max(MIN_LEVEL, min(MAX_LEVEL, job_b_level))

    level_penalty = (
        (job_b_level - job_a_level) * job_b_level if job_b_level > job_a_level else 0
    )
    sector_penalty = 0 if is_same_sector else BETA * job_b_level

    total_penalty = level_penalty + sector_penalty
    score = 100 * (1 - total_penalty / MAX_PENALTY)
    return round(max(0.0, min(100.0, score)), 2)


def get_composite_similarity(
    similarity_score: float,
    task_similarity_score: float,
    nsqf_level_similarity_score: float,
) -> float:
    return (
        similarity_score * SKILL_SIMILARITY_WEIGHTED_AVG
        + task_similarity_score * SKILL_SIMILARITY_WEIGHTED_AVG
        + nsqf_level_similarity_score * NSQF_LEVEL_SIMILARITY_WEIGHTED_AVG
    )


def score_matrix(
    sector1: str,
    sector2: str,
    sub_sector1: Optional[str] = None,
    sub_sector2: Optional[str] = None,
    occupation1: Optional[str] = None,
    occupation2: Optional[str] = None,
    filter_levels: Optional[List[str]] = None,
    filter_technical: Optional[List[str]] = None,
) -> dict:
    sim_matrix = _get_similarity_matrix()

    qp1s = get_filtered_qps(
        sector1, sub_sector1, occupation1, filter_levels, filter_technical
    )
    qp2s = get_filtered_qps(
        sector2, sub_sector2, occupation2, filter_levels, filter_technical
    )

    qp1s_filtered = [qp for qp in qp1s if sim_matrix.has_qp_code(qp._id)]
    qp2s_filtered = [qp for qp in qp2s if sim_matrix.has_qp_code(qp._id)]

    qp1_ids = [qp._id for qp in qp1s_filtered]
    qp2_ids = [qp._id for qp in qp2s_filtered]

    qp_lookup = {qp._id: qp for qp in qp1s_filtered + qp2s_filtered}

    task_data_cache = {}
    for qp_id in set(qp1_ids + qp2_ids):
        task_data = get_qp_task_data(qp_id)
        if task_data:
            task_data_cache[qp_id] = task_data.get("skill_hierarchy")

    scores: List[List[float]] = []

    for qp1_id in qp1_ids:
        qp_scores = []
        lhs_hierarchy = task_data_cache.get(qp1_id)
        for qp2_id in qp2_ids:
            skill_sim_score = get_percentage_match(qp1_id, qp2_id)
            rhs_hierarchy = task_data_cache.get(qp2_id)
            task_sim_score = get_task_similarity(
                qp1_id, qp2_id, lhs_hierarchy, rhs_hierarchy
            )

            lhs_qp = qp_lookup.get(qp1_id)
            rhs_qp = qp_lookup.get(qp2_id)

            nsqf_level_sim_score = 0.0
            if lhs_qp and rhs_qp:
                lhs_level = (
                    lhs_qp.nsqf_level if lhs_qp.nsqf_level is not None else 5.0
                )
                rhs_level = (
                    rhs_qp.nsqf_level if rhs_qp.nsqf_level is not None else 5.0
                )
                is_same_sector = (
                    lhs_qp.sector.id == rhs_qp.sector.id
                    if lhs_qp.sector and rhs_qp.sector
                    else False
                )
                nsqf_level_sim_score = get_nsqf_level_similarity(
                    float(lhs_level), float(rhs_level), is_same_sector
                )

            composite_sim_score = (
                skill_sim_score * SKILL_SIMILARITY_WEIGHTED_AVG
                + task_sim_score * TASK_SIMILARITY_WEIGHTED_AVG
                + nsqf_level_sim_score * NSQF_LEVEL_SIMILARITY_WEIGHTED_AVG
            )

            qp_scores.append(composite_sim_score)
        scores.append(qp_scores)

    qp_metadata: Dict[str, dict] = {}
    for qp in qp1s_filtered + qp2s_filtered:
        qp_metadata[qp._id] = {
            "job_role": qp.job_role,
            "sector_id": qp.sector.id,
            "sector": qp.sector.name,
            "sub_sector": (
                qp.sector.sub_sectors[0].name if qp.sector.sub_sectors else None
            ),
            "sub_sector_id": (
                qp.sector.sub_sectors[0].id if qp.sector.sub_sectors else None
            ),
            "occupation_id": qp.occupation.id,
            "occupation": qp.occupation.description,
            "nsqf_level": qp.nsqf_level,
            "technical": qp.technical,
        }

    return {
        "qp1": qp1_ids,
        "qp2": qp2_ids,
        "similarity_scores": scores,
        "metadata": qp_metadata,
    }


# ------------------------------------------------------------------------------------
# CLI commands (pair & matrix)
# ------------------------------------------------------------------------------------


def _save_or_print(payload: dict[str, Any], out_path: Path | None) -> None:
    if out_path:
        out_path.parent.mkdir(parents=True, exist_ok=True)
        out_path.write_text(json.dumps(payload, indent=2), encoding="utf-8")
        print(f"Wrote results to {out_path}")
    else:
        print(json.dumps(payload, indent=2))


def build_parser() -> argparse.ArgumentParser:
    parser = argparse.ArgumentParser(description="QP similarity tooling.")
    parser.add_argument(
        "--output",
        type=Path,
        help="Optional path to dump the JSON response.",
    )

    subparsers = parser.add_subparsers(dest="command", required=True)

    pair_parser = subparsers.add_parser(
        "pair",
        help="Replicates the /compare-qps API endpoint for two QPs.",
    )
    pair_parser.add_argument("lhs_qp_id", help="Left-hand QP ID (e.g. AGR_Q0509_1.0).")
    pair_parser.add_argument("rhs_qp_id", help="Right-hand QP ID.")

    matrix_parser = subparsers.add_parser(
        "matrix",
        help="Replicates the /qp-similarity-scores API endpoint.",
    )
    matrix_parser.add_argument("--sector1", required=True, help="First sector ID.")
    matrix_parser.add_argument("--sector2", required=True, help="Second sector ID.")
    matrix_parser.add_argument(
        "--sub-sector1", dest="sub_sector1", help="Optional sub-sector ID for sector1."
    )
    matrix_parser.add_argument(
        "--sub-sector2", dest="sub_sector2", help="Optional sub-sector ID for sector2."
    )
    matrix_parser.add_argument("--occupation1", help="Optional occupation ID for sector1.")
    matrix_parser.add_argument("--occupation2", help="Optional occupation ID for sector2.")
    matrix_parser.add_argument(
        "--filter-levels",
        nargs="*",
        help="Restrict to NSQF levels (space-separated).",
    )
    matrix_parser.add_argument(
        "--filter-technical",
        nargs="*",
        help='Restrict to Technical/Non-Technical. Example: --filter-technical Technical "Non-Technical"',
    )

    return parser


def main() -> None:
    parser = build_parser()
    args = parser.parse_args()
    if args.command == "pair":
        payload = run_pair_command(args.lhs_qp_id, args.rhs_qp_id)
    elif args.command == "matrix":
        payload = run_matrix_command(
            sector1=args.sector1,
            sector2=args.sector2,
            sub_sector1=args.sub_sector1,
            sub_sector2=args.sub_sector2,
            occupation1=args.occupation1,
            occupation2=args.occupation2,
            filter_levels=args.filter_levels,
            filter_technical=args.filter_technical,
        )
    else:  # pragma: no cover
        parser.error("Unknown command")
        return

    _save_or_print(payload, args.output)


if __name__ == "__main__":
    main()


Data quality

In [ ]:
import pandas as pd
import json
import io

input_filename = 'qp_listings_current_jobs.csv'
output_filename = 'updated_qp_listings_current_jobs.csv'

try:
    # 'sep=None' with 'engine=python' tells pandas to sniff the separator automatically
    df = pd.read_csv(input_filename, sep=None, engine='python')

    # Clean column headers: removes leading/trailing spaces (e.g. " qpParamOne " -> "qpParamOne")
    df.columns = df.columns.str.strip()

    print("Columns detected:", df.columns.tolist())

    if 'qpParamOne' in df.columns:
        # Define the cleaning function
        def clean_qp_param(value):
            try:
                if isinstance(value, str) and value.strip().startswith('{'):
                    data = json.loads(value)
                    return data.get('paramDesc', value)
            except (json.JSONDecodeError, TypeError):
                pass
            return value

        # Apply the cleaning
        df['qpParamOne'] = df['qpParamOne'].apply(clean_qp_param)

        # Save the file (using tab separator to match your original structure)
        df.to_csv(output_filename, index=False, sep='\t')
        print(f"Success! Updated file saved to: {output_filename}")

        # safely print preview
        cols_to_show = [c for c in ['qpCode', 'qpParamOne'] if c in df.columns]
        print(df[cols_to_show].head())

    else:
        print("\nERROR: Could not find 'qpParamOne' column even after auto-detection.")
        print("Please check if your CSV headers match the spelling exactly.")

except FileNotFoundError:
    print(f"Error: The file '{input_filename}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Columns detected: ['qpCode', 'version', 'nsqfLevel', 'jobRole', 'jobRoleDesc', 'qpParamOne', 'nqrCode', 'qp_code', 'qp_version', 'qp_name', 'qp_path', 'sectorID', 'Sector Name', 'subSectorID', 'subSectorName', 'Correct Sector Matched', 'Tags', 'Subsectors added or not', 'matched_filename']
Success! Updated file saved to: updated_qp_listings_current_jobs.csv
      qpCode     qpParamOne
0  IAS/Q3001      Technical
1  HYC/Q9101      Technical
2  RSC/Q0831      Technical
3  BSC/Q2401  Non-Technical
4  ASC/Q1402      Technical


In [ ]:
import pandas as pd

# 1. Setup filenames
input_filename = 'updated_qp_listings_current_jobs.csv' # The file from the previous step
output_filename = 'duplicate_job_roles_sheet.csv'

try:
    # Load the file (auto-detects if it uses tabs or commas)
    df = pd.read_csv(input_filename, sep=None, engine='python')

    # Clean column headers (removes hidden spaces)
    df.columns = df.columns.str.strip()

    if 'jobRole' in df.columns:
        # 2. Find duplicates
        # keep=False ensures we capture ALL rows that have a duplicate (e.g., both "a" and "a")
        duplicate_mask = df.duplicated(subset=['jobRole'], keep=False)
        df_duplicates = df[duplicate_mask]

        # 3. Sort them so identical roles are grouped together
        # (e.g., a,a,a, then g,g, then s,s,s)
        df_duplicates_sorted = df_duplicates.sort_values(by='jobRole')

        # 4. Report statistics
        total_dup_rows = len(df_duplicates_sorted)
        unique_dup_roles = df_duplicates_sorted['jobRole'].nunique()

        print("-" * 30)
        print("DUPLICATE REPORT")
        print("-" * 30)
        print(f"Total rows shared by duplicates: {total_dup_rows}")
        print(f"Number of unique Job Roles repeated: {unique_dup_roles}")

        if total_dup_rows > 0:
            print("\nMost frequent duplicates:")
            print(df_duplicates_sorted['jobRole'].value_counts().head())

        # 5. Save the grouped sheet
        # Using sep='\t' to match your format (change to ',' if you prefer standard CSV)
        df_duplicates_sorted.to_csv(output_filename, index=False, sep='\t')
        print(f"\nSaved grouped duplicates to: {output_filename}")

    else:
        print("Error: Column 'jobRole' not found in the CSV.")

except FileNotFoundError:
    print(f"Error: The file '{input_filename}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

------------------------------
DUPLICATE REPORT
------------------------------
Total rows shared by duplicates: 515
Number of unique Job Roles repeated: 257

Most frequent duplicates:
jobRole
Water Pump Operator                                           3
Junior Instrumentation Technician (Process Control)           2
Junior Mechanic (Electrical/ Electronics/ Instrumentation)    2
Junior Mechanic (Engine)                                      2
Junior Rubber Technician/Technical Assistant (Rubber)         2
Name: count, dtype: int64

Saved grouped duplicates to: duplicate_job_roles_sheet.csv


In [ ]:
import pandas as pd

# 1. Load the parquet file
file_path = "all_similarities_final.parquet"
try:
    df = pd.read_parquet(file_path)

    # 2. Apply the filter mentioned in the feedback
    # "rows with duplicate job roles (i.e., same job role name in 1 and 2)"
    same_role_mask = df['jobRole_1'] == df['jobRole_2']

    # 3. Refine the filter to find the "confusing" cases
    # The feedback says: "Although their QP codes are different, everything else is same."
    # So we filter for Same Role AND Different QP Code
    diff_qp_mask = df['qp_code_1'] != df['qp_code_2']

    # Combine masks to get the target rows
    target_rows = df[same_role_mask & diff_qp_mask]

    print(f"Total rows with same Job Role: {same_role_mask.sum()}")
    print(f"Rows with same Job Role but DIFFERENT QP Code: {len(target_rows)}")

    # 4. Save the result to a CSV file for manual inspection
    output_file = "duplicate_roles_diff_qp.csv"
    target_rows.to_csv(output_file, index=False)
    print(f"\nFiltered rows saved to: {output_file}")

    # 5. Quick Analysis: Check if 'everything else' is actually the same
    # We can inspect a few columns to see where the differences lie
    if not target_rows.empty:
        print("\nSample of QP Code pairs for these duplicates:")
        print(target_rows[['qp_code_1', 'qp_code_2', 'jobRole_1']].head(10))

        # Check if NSQF Levels are also same
        nsqf_diff = target_rows[target_rows['nsqfLevel_1'] != target_rows['nsqfLevel_2']]
        print(f"\nNumber of these rows where NSQF Level is DIFFERENT: {len(nsqf_diff)}")

        # Check if Sector Names are also same
        sector_diff = target_rows[target_rows['Sector Name_1'] != target_rows['Sector Name_2']]
        print(f"Number of these rows where Sector Name is DIFFERENT: {len(sector_diff)}")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Total rows with same Job Role: 259
Rows with same Job Role but DIFFERENT QP Code: 259

Filtered rows saved to: duplicate_roles_diff_qp.csv

Sample of QP Code pairs for these duplicates:
       qp_code_1                      qp_code_2  \
6662   RSC/Q0831          2022/RUB/RCPSDC/06938   
13065  ASC/Q3503            2022/AUT/ASDC/06567   
15785  ASC/Q1001    QG-04-AU-03592-2025-V2-ASDC   
19854  ISC/Q0904  QG-03-IS-03888-2025-V2-IISSSC   
22435  ASC/Q3103            2022/AUT/ASDC/06569   
24382  ISC/Q0906  QG-03-IS-03885-2025-V2-IISSSC   
36127  PSC/Q0102    QG-03-PL-03440-2024-V2-WMPS   
38273  PSC/Q0104    QG-04-PL-03441-2024-V2-WMPS   
47337  THC/Q0109   QG-4.5-TH-02014-2024-V1-THSC   
49210  ISC/Q0410  QG-02-IS-03876-2025-V2-IISSSC   

                                               jobRole_1  
6662   Junior Rubber Technician/Technical Assistant (...  
13065                Automotive CNC Machining Technician  
15785                         Automotive Sales Executive  
19854           

In [ ]:
import pandas as pd
import json

# 1. Load the datasets
# Replace with the actual paths if they are in a specific folder in your Drive
df_a = pd.read_csv('adb-share-intermediate.csv')
df_b = pd.read_csv('qp_listings_current_jobs.csv')

# Optional: Ensure the key column 'qp_code' is the same data type in both (e.g., string)
# This prevents errors if one is read as a number and the other as text
df_a['qp_code'] = df_a['qp_code'].astype(str)
df_b['qp_code'] = df_b['qp_code'].astype(str)

# ---------------------------------------------------------
# STEP 1: Merge columns from 'a' into 'b'
# ---------------------------------------------------------

# We filter df_a to only keep the columns we need + the joining key
cols_to_fetch = ['qp_code', 'Source', 'skill_hierarchy']
subset_a = df_a[cols_to_fetch]

# Perform a LEFT merge.
# This keeps all rows from 'b' and adds matching info from 'a'.
merged_df = pd.merge(df_b, subset_a, on='qp_code', how='left')

# ---------------------------------------------------------
# STEP 2: Update and Rename 'qpParamOne'
# ---------------------------------------------------------

def extract_param_desc(row_data):
    """
    Parses the JSON string and extracts the 'paramDesc' value.
    Returns 'Unknown' or the original value if parsing fails.
    """
    try:
        # Check if data is valid
        if pd.isna(row_data):
            return None

        # If it's already a dictionary (unlikely in CSV but possible in some formats)
        if isinstance(row_data, dict):
            return row_data.get('paramDesc')

        # Parse the string as JSON
        data_dict = json.loads(row_data)
        return data_dict.get('paramDesc')

    except (json.JSONDecodeError, AttributeError):
        # Fallback if the string is malformed
        return row_data

# Apply the function to the column
merged_df['qpParamOne'] = merged_df['qpParamOne'].apply(extract_param_desc)

# Rename the column
merged_df.rename(columns={'qpParamOne': 'Technical/Non-Technical'}, inplace=True)

# ---------------------------------------------------------
# STEP 3: Save the result
# ---------------------------------------------------------

# Display the first few rows to verify
print("Preview of updated data:")
print(merged_df[['qp_code', 'Technical/Non-Technical', 'Source', 'skill_hierarchy']].head())

# Save to a new CSV file
merged_df.to_csv('qp_listings_updated.csv', index=False)
print("\nSuccess! File saved as 'qp_listings_updated.csv'")

Preview of updated data:
     qp_code Technical/Non-Technical Source  \
0  IAS_Q3001               Technical   NSQP   
1  HYC_Q9101               Technical   NSQP   
2  RSC_Q0831               Technical   NSQP   
3  BSC_Q2401           Non-Technical   NSQP   
4  ASC_Q1402               Technical   NSQP   

                                     skill_hierarchy  
0  [{"l1id": "m196", "norm_w_sum_of_all_tasks_und...  
1  [{"l1id": "m120", "norm_w_sum_of_all_tasks_und...  
2  [{"l1id": "m52", "norm_w_sum_of_all_tasks_unde...  
3  [{"l1id": "m83", "norm_w_sum_of_all_tasks_unde...  
4  [{"l1id": "m279", "norm_w_sum_of_all_tasks_und...  

Success! File saved as 'qp_listings_updated.csv'


In [ ]:
import pandas as pd

# 1. Load the files
# Load the file we created in the previous step
df_b = pd.read_csv('/content/qp_listings_updated.csv')

# Load the new file containing 'nco_clean'
# Make sure the filename matches exactly what you uploaded
df_new_source = pd.read_csv('A_jr_jd_nc_qpc_updated_with_skills.csv')

# 2. Prepare the data
# Ensure the key column 'qp_code' is the same data type (string) in both
df_b['qp_code'] = df_b['qp_code'].astype(str)
df_new_source['qp_code'] = df_new_source['qp_code'].astype(str)

# 3. Merge the specific column
# We create a subset with only 'qp_code' (for matching) and 'nco_clean' (to add)
# This prevents accidental duplication of other columns
cols_to_fetch = ['qp_code', 'nco_clean']
subset_source = df_new_source[cols_to_fetch]

# Perform a LEFT merge to keep all rows from your listings file
# and just add the matching nco_clean data
df_final = pd.merge(df_b, subset_source, on='qp_code', how='left')

# 4. Save the final result
print("Merge successful. Preview of new column:")
print(df_final[['qp_code', 'nco_clean']].head())

# Save to a new csv
df_final.to_csv('qp_listings_final_with_nco.csv', index=False)
print("\nFile saved as 'qp_listings_final_with_nco.csv'")

Merge successful. Preview of new column:
     qp_code nco_clean
0  IAS_Q3001   3111.99
1  HYC_Q9101   7212.03
2  RSC_Q0831   4322.02
3  BSC_Q2401   3312.01
4  ASC_Q1402   3115.06

File saved as 'qp_listings_final_with_nco.csv'
